In [1]:
import os 
from sqlalchemy import create_engine
import pandas as pd

In [2]:
user1 = os.environ['PAYMEE_SONTATA_DB_USER']
password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
db = "Sonata_Connect"
server = "172.17.130.216"   

engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")


In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
sql_query = f"""SET NOCOUNT ON;SELECT task_id, task_description, status, priority, assigned_to, created_by, completion_date, created_on, deadline_date, category, review, feedback FROM [{db}].[dbo].accounts_task_details ORDER BY created_on;SET NOCOUNT OFF"""
all_tasks_df=pd.read_sql_query(sql_query,engine)


sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 6892;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)


In [5]:
all_tasks_df.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None


In [6]:
df

,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
0,1,1,0,Head Office,6892,BRIJESH KUMAR VISHWAKARMA,1,41,IT DO
1,650,2,1,Prayagraj Division,7540,ANSHUMAN TIWARI,650,39,Disbursement Confirmer
2,650,2,1,Prayagraj Division,9739,LAKSHMAN SINGH,650,34,DIVISION HEAD
3,650,2,1,Prayagraj Division,9739,LAKSHMAN SINGH,650,44,Credit Committee
4,650,2,1,Prayagraj Division,15183,ABDUL KADIR ANSARI,650,58,Audit
...,...,...,...,...,...,...,...,...,...
3006,2326,5,2206,Chandwa,27375,Rahul Raj,2326,13,Branch Manager
3007,2326,5,2206,Chandwa,31817,MODASSHIR RAZA,2326,55,BRO
3008,2327,5,607,Kurha,8699,SHASHIDHAR PASWAN,2327,13,Branch Manager
3009,2327,5,607,Kurha,31763,SAURABH KUMAR,2327,55,BRO


In [7]:
reqdf = pd.merge(all_tasks_df, df, how='left', left_on='assigned_to', right_on='UserID').drop_duplicates(subset=['task_id'])
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO


In [8]:
len(reqdf)

48908

In [9]:
div_df = df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = df[(df['RoleId']==35)&(df['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
hub_df = df[(df['RoleId']==36)&(df['RoleName']=='HUB HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName'}, inplace=True)
bm_df = df[(df['RoleId']==13)&(df['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = df[(df['RoleId']==55)&(df['RoleName']=='BRO')][['U_BUID','UserID','UserName']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

In [10]:
dF.head()

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,591,4829,ANKIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3353,RAHUL KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
2,2,25715,DHEERAJ KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
3,2,27274,Saurabh Kushwaha,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division
4,2,31073,NEERAJ KUMAR,2.0,Sirathu,4797.0,PINTU KUMAR YADAV,545.0,545.0,1692.0,Shivlal pal,BHARWARI,2180.0,2180.0,18044.0,KAMLESH YADAV,Kaushambi,650.0,650.0,9739.0,LAKSHMAN SINGH,Prayagraj Division


In [11]:
reqdf.sample()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName
1469,130585,Customer (Customer id: Dmh:Dmh:01:329:021) has...,Pending,1,27081,19119,None,2023-04-03,2023-04-03,10,None,None,190.0,5.0,567.0,Damoh,27081.0,Shubham Patel,190.0,55.0,BRO


In [12]:
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )

In [13]:
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,4172.0,Akash Kumar Sen,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division


In [22]:
bm_df

,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId
176,2212,Giridih,21376,Rajesh Kumar Yadav,2200
362,2,Sirathu,4797,PINTU KUMAR YADAV,545
366,3,Nari Bari,2426,RAJ KAMAL,2222
371,4,Akbarpur,2164,Anoop Singh,551
379,5,Tanda,3387,Krishna Kumar,551
...,...,...,...,...,...
3001,2324,Aadhartal,26898,Ruby Dahiya,562
3004,2325,Nirmali,25630,Dharmveer Kumar,2104
3006,2326,Chandwa,27375,Rahul Raj,2206
3008,2327,Kurha,8699,SHASHIDHAR PASWAN,607


In [14]:
len(dF)

2099

In [24]:
all_tasks_df

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
48903,192839,Customer (Customer id: 2918833) will pay an am...,Pending,1,30737,2550,None,2023-04-28,2023-04-30,3,None,None
48904,192840,Customer (Customer id: 791890) will pay an amo...,Pending,1,31191,31191,None,2023-04-28,2023-05-10,3,None,None
48905,192841,Customer (Customer id: 2562161) will pay an am...,Pending,1,28450,28450,None,2023-04-28,2023-05-03,3,None,None
48906,192842,Customer (Customer id: 1456849) will pay an am...,Pending,1,31327,27586,None,2023-04-28,2023-04-29,3,None,None


In [16]:
reqdf = pd.merge(reqdf, dF, how='left', left_on='UserID', right_on = 'BRO_UserID' )

In [17]:
reqdf.head()

,task_id,task_description,status,priority,assigned_to,created_by,completion_date,created_on,deadline_date,category,review,feedback,BUId,BUType,ReportingBUId,buname,UserID,UserName,U_BUID,RoleId,RoleName,BRO_BUID_x,BRO_UserID_x,BRO_UserName_x,BM_BUID_x,BranchName_x,BM_UserID_x,BM_UserName_x,BM_ReportingBUId_x,Hub_BUID_x,Hub_UserID_x,Hub_UserName_x,HubName_x,Hub_ReportingBUId_x,Region_BUID_x,Region_UserID_x,Region_UserName_x,RegionName_x,Region_ReportingBUId_x,Div_BUID_x,Div_UserID_x,Div_UserName_x,DivName_x,BRO_BUID_y,BRO_UserID_y,BRO_UserName_y,BM_BUID_y,BranchName_y,BM_UserID_y,BM_UserName_y,BM_ReportingBUId_y,Hub_BUID_y,Hub_UserID_y,Hub_UserName_y,HubName_y,Hub_ReportingBUId_y,Region_BUID_y,Region_UserID_y,Region_UserName_y,RegionName_y,Region_ReportingBUId_y,Div_BUID_y,Div_UserID_y,Div_UserName_y,DivName_y
0,129116,Customer (Customer id: Sdl:Bhr:01:006) is inte...,Completed,1,29938,2271,2023-04-21,2023-04-01,2023-04-01,7,None,None,459.0,5.0,2141.0,Burhar,29938.0,Virendra Singh gound,459.0,55.0,BRO,459.0,29938.0,Virendra Singh gound,459.0,Burhar,4172.0,Akash Kumar Sen,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur,459.0,29938.0,Virendra Singh gound,459.0,Burhar,4172.0,Akash Kumar Sen,2141.0,2141.0,18022.0,Deepak Jagati,Burhar,2183.0,2183.0,1622.0,Pradeep kachhi,Katni,652.0,652.0,15190.0,JITENDRA YADAV,Jabalpur
1,129117,Customer (Customer id: Rab:cit:01:236:037) has...,Pending,1,30482,27199,None,2023-04-01,2023-04-01,10,None,None,79.0,5.0,584.0,Raibareli,30482.0,DHARMENDRA BAHADUR,79.0,55.0,BRO,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division,79.0,30482.0,DHARMENDRA BAHADUR,79.0,Raibareli,1884.0,Vinod Kumar,584.0,584.0,15051.0,DHARMENDRA PRATAP SINGH,Raibarely,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
2,129118,Customer (Customer id: Bab:cit:01:189:043) has...,Pending,1,27245,27199,None,2023-04-01,2023-04-01,10,None,None,8.0,5.0,583.0,Barabanki city,27245.0,ROLI KUMARI,8.0,55.0,BRO,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division,8.0,27245.0,ROLI KUMARI,8.0,Barabanki city,3682.0,BHUPENDRA SINGH,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division
3,129119,Customer (Customer id: 1397268) will pay an am...,Pending,1,27815,27815,None,2023-04-01,2023-04-01,3,None,None,424.0,5.0,2139.0,Guna,27815.0,AAKASH YADAV,424.0,55.0,BRO,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior,424.0,27815.0,AAKASH YADAV,424.0,Guna,2702.0,Prahlad Ahirwar,2139.0,2139.0,259.0,Vikash Namdev,GUNA,1013.0,1013.0,16347.0,Susheel Kumar Yadav,Gwalior,2156.0,2156.0,10851.0,Vinod singh,Gwalior
4,129120,Customer (Customer id: Lko:mhn:01:116:004) has...,Pending,1,31063,27199,None,2023-04-01,2023-04-01,10,None,None,184.0,5.0,583.0,Mohanlalganj,31063.0,SATYAM SINGH,184.0,55.0,BRO,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division,184.0,31063.0,SATYAM SINGH,184.0,Mohanlalganj,21.0,MOHD IRFAN,583.0,583.0,3245.0,Krishan Kumar Yadav,Barabanki,1002.0,1002.0,0.0,REGION HEAD NOT POSTED,Lucknow,651.0,651.0,7535.0,RAKESH KUMAR CHAUDHARY,Lucknow Division


In [18]:
cols_req = [
    'created_on',
    'DivName',
    'RegionName',
    'HubName',
    'BranchName',
    'assigned_to',
    'created_by',
    'priority',
    'task_id',
    'category',
    'task_description',
    'deadline_date',
    #reschedule date
    #whatsapp conf
    'status',
    'completion_date',
    'review',
    'feedback'
    ]

In [19]:
reqdf.columns

Index(['task_id', 'task_description', 'status', 'priority', 'assigned_to',
       'created_by', 'completion_date', 'created_on', 'deadline_date',
       'category', 'review', 'feedback', 'BUId', 'BUType', 'ReportingBUId',
       'buname', 'UserID', 'UserName', 'U_BUID', 'RoleId', 'RoleName',
       'BRO_BUID_x', 'BRO_UserID_x', 'BRO_UserName_x', 'BM_BUID_x',
       'BranchName_x', 'BM_UserID_x', 'BM_UserName_x', 'BM_ReportingBUId_x',
       'Hub_BUID_x', 'Hub_UserID_x', 'Hub_UserName_x', 'HubName_x',
       'Hub_ReportingBUId_x', 'Region_BUID_x', 'Region_UserID_x',
       'Region_UserName_x', 'RegionName_x', 'Region_ReportingBUId_x',
       'Div_BUID_x', 'Div_UserID_x', 'Div_UserName_x', 'DivName_x',
       'BRO_BUID_y', 'BRO_UserID_y', 'BRO_UserName_y', 'BM_BUID_y',
       'BranchName_y', 'BM_UserID_y', 'BM_UserName_y', 'BM_ReportingBUId_y',
       'Hub_BUID_y', 'Hub_UserID_y', 'Hub_UserName_y', 'HubName_y',
       'Hub_ReportingBUId_y', 'Region_BUID_y', 'Region_UserID_y',
       'Reg

In [ ]:
from datetime import datetime, date

In [ ]:
for row in reqdf.head().itertuples():
    print(row)

In [ ]:
context = {}
context['r'] = reqdf

In [ ]:
context['r']

In [ ]:
d = dict(zip(df['UserID'], df['UserName']))


In [ ]:
reqdfcopy = reqdf.copy()

In [ ]:
reqdfcopy['assigned_to'].apply(lambda x : str(x) + '-' + d[x] if x in d.keys() else str(x))

In [ ]:
dlist=df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')]['buname'].to_list()
df['userid_buid'] = df[df['RoleId']==34]['UserID'].astype(str) +"_"+ df[df['RoleId']==34]['U_BUID'].astype(str)
df2 = df.dropna(subset=['userid_buid'], how='all')
buid_list = df2['userid_buid'].to_list()
ddict= dict(zip(buid_list,dlist))
ddict


In [ ]:
from_date = datetime.date(datetime.strptime(str('2023-04-10'), '%Y-%m-%d'))
to_date = datetime.date(datetime.strptime(str('2023-04-12'), '%Y-%m-%d'))

In [37]:
sagar_bro_list = dF[dF['RegionName'] == 'Sagar']['BRO_UserID'].to_list()

In [38]:
sagar_bro_list

[27674,
 29702,
 30515,
 31337,
 31766,
 32102,
 32103,
 25125,
 27862,
 29808,
 29809,
 30766,
 31517,
 31661,
 31903,
 28330,
 30304,
 30688,
 30769,
 31243,
 31491,
 31622,
 27801,
 28177,
 29921,
 31106,
 31129,
 31208,
 31519,
 5050,
 25685,
 28356,
 30724,
 25614,
 27081,
 27675,
 27983,
 30031,
 31251,
 18011,
 29879,
 30373,
 30493,
 30962,
 31845,
 32104,
 28256,
 29884,
 29885,
 30765,
 31417,
 31673,
 4664,
 22614,
 31080,
 31463,
 27681,
 30299,
 30983,
 31164,
 18049,
 31196,
 31349,
 31641,
 32115,
 32131,
 23764,
 25661,
 27289,
 29742,
 29980,
 25356,
 25786,
 29594,
 30049,
 30661,
 31628,
 29831,
 30296,
 31520,
 31740,
 31847,
 31901,
 32148,
 32149,
 25470,
 30812,
 30835,
 31000,
 29670,
 30608,
 30706,
 31741,
 31765,
 31902,
 28257,
 30924,
 30979]

In [102]:


user1 = os.environ['PAYMEE_SONTATA_DB_USER']
password = os.environ['PAYMEE_SONTATA_DB_PASSWORD']
db = "Sonata_Connect"
server = "172.17.130.216"

engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[S_SP_BusinessCallingCardData] @bcd_Userid = 27199,@bcd_id = 10 ;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)

In [103]:
df[(df['BRO_UserID'] == 31063) & (df['AllocatedDate'] == '2023-05-01')]

,DPD_CATG_String,DPD_CATG_Classification,DisbursementID,id,feedback_code,calling_attempt,IS_active,AllocatedDate,UserID,Centercode,CustomerInfoId,latest_collected_date,user_arrear_max,current_installment_ID,pending_amount,user_dpd_max,DisbursedAmt,DisbursementDate,AVERAGE_ACCT_AGE,ProductName,ApplicantName,mobile_no,Loan_Closing_Date,Other_Active_Accounts,current_installment_Amount,upcoming_installment_ID,upcoming_installment_Amount,Others_Last_Paid_Date,Sonata_Last_Paid_Date,loan_classification,CenterMeetingDay,CenterMeetingTime,CenterName,CustomerCode,DivisionID,Division_BuName,RegionID,Region_BuName,HubID,HUB_BuName,BranchID,Branch_BuName,BRO_UserID,BRO_Name,BRO_ContactNo,BM_UserID,BM_Name,BM_ContactNo
153,None,None,6918446,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:138,2540146.0,2023-01-23,5.0,44.0,4000.0,63.0,33840.0,2021-08-02,1yrs 11mon,Fortnightly Loan (33840),Vimala,9648444176,2023-08-28,0.0,800.0,53.0,800.0,1900-01-01,2022-10-17,AR,55.0,1900-01-01 11:30:00,Rajeevnagar4,Lko:mhn:01:050:016,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
242,0000001,First time arrear,6940916,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:138,2762081.0,2023-04-17,6.0,43.0,7501.0,77.0,63450.0,2021-08-19,1yrs 1mon,Fortnightly Loan (63450),Pushpa Devi Thapa,7607326667,2023-09-11,0.0,1500.0,53.0,1500.0,1900-01-01,2022-10-10,AR,55.0,1900-01-01 11:30:00,Rajeevnagar4,Lko:mhn:01:050:022,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
398,None,None,6971766,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:135,1475718.0,2023-04-04,6.0,42.0,8100.0,83.0,63450.0,2021-09-02,1yrs 5mon,Fortnightly Loan (63450),Anjali,7355040294,2023-09-19,5.0,1500.0,53.0,1500.0,2022-10-28,2022-11-01,AR,56.0,1900-01-01 11:30:00,Nilmatha,Luc:Bar:184:135:012,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
608,None,None,7045496,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:135,2738638.0,2023-01-24,6.0,40.0,7200.0,83.0,50760.0,2021-09-29,0yrs 10mon,Fortnightly Loan (50760),Neetu Singh,9670461614,2023-10-17,1.0,1200.0,53.0,1200.0,2020-07-17,2022-11-01,AR,56.0,1900-01-01 11:30:00,Nilmatha,Luc:Bar:184:135:021,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
653,None,None,7056132,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:135,2608735.0,2023-04-04,5.0,40.0,5000.0,62.0,42300.0,2021-10-08,1yrs 2mon,Fortnightly Loan (42300),Sunita Devi,9336267083,2023-10-24,2.0,1000.0,53.0,1000.0,2022-10-28,2022-10-04,AR,56.0,1900-01-01 11:30:00,Nilmatha,Luc:Bar:184:135:019,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
841,None,None,7119783,10,None,0,True,2023-05-01,31063.0,Luc:Mal:184:145,2678684.0,2023-04-27,5.0,33.0,4258.0,67.0,43260.0,2022-01-08,1yrs 0mon,Ashirvad CGS Fortnightly (43260),Madhu Devi,8419817337,2024-01-25,1.0,1000.0,53.0,1000.0,2022-10-01,2022-10-20,AR,58.0,1900-01-01 10:46:00,Topkhana,Luc:Mal:184:145:010,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
1035,None,None,7249726,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:134,3229853.0,2023-03-13,6.0,28.0,4800.0,84.0,26345.0,2022-03-30,1yrs 8mon,Fortnightly (26345),Gudiya Dhanu,8953015355,2023-10-02,0.0,800.0,39.0,800.0,2022-06-29,2022-10-10,AR,55.0,1900-01-01 10:30:00,RAJEEVNAGAR,Lko:mhn:01:050:029,651.0,Lucknow Division,1002,Lucknow,583,Barabanki,184,Mohanlalganj,31063.0,SATYAM SINGH,9696344756,21.0,MOHD IRFAN,8400333748
1059,None,None,7257806,10,None,0,True,2023-05-01,31063.0,Luc:Bar:184:134,2629362.0,2023-04-24,5.0,27.0,6000.0,63.0,50860.0,2022-04-06,1yrs 3mon,JLG Fortnightly (50860),Naziya Bano,8953684878,2024-04-22,2.0,1200.0,53.0,1200.0,2022-10-20,2022-10-31,AR,55.

In [92]:
sagarreg = dF[dF['RegionName'] == 'Sagar']

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
71,24,27674,SONAM VISHWAKARMA,24.0,Garhakota,25157.0,Rukhsar Khan,567.0,567.0,14045.0,Arvind tiwari,DAMOH,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
72,24,29702,SANGEETA,24.0,Garhakota,25157.0,Rukhsar Khan,567.0,567.0,14045.0,Arvind tiwari,DAMOH,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
73,24,30515,SEETA RAM LODHI,24.0,Garhakota,25157.0,Rukhsar Khan,567.0,567.0,14045.0,Arvind tiwari,DAMOH,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
74,24,31337,SHIVA SEN,24.0,Garhakota,25157.0,Rukhsar Khan,567.0,567.0,14045.0,Arvind tiwari,DAMOH,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
75,24,31766,RAJA RAM AHIRWAR,24.0,Garhakota,25157.0,Rukhsar Khan,567.0,567.0,14045.0,Arvind tiwari,DAMOH,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,401,31765,MANISH CHADAR,401.0,Shahgarh,21254.0,Arvind Lodhi,2162.0,2162.0,4166.0,Suraj Singh lodhi,Shahgarh,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
1198,401,31902,SANJAY YADAV,401.0,Shahgarh,21254.0,Arvind Lodhi,2162.0,2162.0,4166.0,Suraj Singh lodhi,Shahgarh,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
2020,2277,28257,AMAN SAHU,2277.0,Vidisha,17919.0,Dharmendra Patel,2276.0,2276.0,1859.0,Anil Choudhery,Vidisha,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior
2021,2277,30924,ROHIT GADARIYA,2277.0,Vidisha,17919.0,Dharmendra Patel,2276.0,2276.0,1859.0,Anil Choudhery,Vidisha,1015.0,1015.0,18024.0,Shailendra Rajput,Sagar,2156.0,2156.0,10851.0,Vinod singh,Gwalior


In [105]:
dF[dF['Region_UserID'] == 3557]

,BRO_BUID,BRO_UserID,BRO_UserName,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName


In [109]:
engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[S_SP_Retention_Calling_Card_Data] @bcd_Userid = 27199;SET NOCOUNT OFF"""
rcdf=pd.read_sql_query(sql_query,engine)

In [111]:
rcdf[rcdf['Centercode'] == 'Lko:mab:02:018']

,DPD_CATG_String,DPD_CATG_Classification,DisbursementID,CustomerInfoID,feedback_code,calling_attempt,IS_active,UserID,AllocatedDate,Centercode,CustomerInfoId,latest_collected_date,user_arrear_max,current_installment_ID,pending_amount,user_dpd_max,DisbursedAmt,DisbursementDate,SonataLoanCnt,InstallmentsInNPA,Other_Activ_Accounts,ProductName,ApplicantName,mobile_no,Loan_Closing_Date,current_installment_Amount,upcoming_installment_ID,upcoming_installment_Amount,Sonata_Last_Paid_Date,loan_classification,CenterMeetingDay,CenterMeetingTime,CenterName,CustomerCode,LoanType,Is_NPA_Before,DivisionID,Division_BuName,RegionID,Region_BuName,HubID,HUB_BuName,BranchID,Branch_BuName,BRO_UserID,BRO_Name,BRO_ContactNo,BM_UserID,BM_Name,BM_ContactNo,Total_Risk_Score,Total_Risk
59,,,7050632,3125626,None,None,True,27199,2023-05-01,Lko:mab:02:018,3125626,2023-04-06,0,0,0,0,26455.0,2021-10-05,1,0,0,Weekly Loan (26455),Suman,9936308056,2023-04-06,400.0,0,0,2023-04-06,,58.0,1900-01-01 08:00:00,Ramchaura,Lko:mab:02:018:154,49,0,651.0,Lucknow Division,1002,Lucknow,2174,Malihabad,216,Banthra,16469,NAPENDRA KUMAR SHUKLA,8423879983,1104,Rajendra Prajapati,8009478560,100.0,Very Low
78,,,7300104,807991,None,None,True,27199,2023-05-01,Lko:mab:02:018,807991,2023-04-27,0,0,0,0,11040.0,2022-05-09,5,0,1,Nokia C 30 Weekly (11040),punam,8953044147,2023-04-27,250.0,0,0,2023-04-27,,58.0,1900-01-01 08:00:00,Ramchaura,Lko:mab:02:018:018,49,0,651.0,Lucknow Division,1002,Lucknow,2174,Malihabad,216,Banthra,16469,NAPENDRA KUMAR SHUKLA,8423879983,1104,Rajendra Prajapati,8009478560,100.0,Very Low


In [ ]:
import traceback